# NBA Machine Learning

The objective for our project will be to create a deep learning model that will be able to predict future outcomes of a basketball season. 
This is done because this could be used as a viable tool in the process of sports betting. The model would function almost exactly the same as one that predicts stock price and signals when to buy or sell, the only difference is that it would predict a player's stat and would signal when to bet instead of buying/selling.

Our project will use historical data from NBA teams such as previous season's records, shot making percentages, defensive ratings, 
and several others from the last ten seasons in a deep learning model to predict the next season's standings for each season and individual player statistics.


In [80]:
#import libraries required to run program
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [81]:
#import csv containing Jokic's statistics
jokic_csv_path = Path("Resources/jokic_ultimate.csv")
jokic_df = pd.read_csv(jokic_csv_path)

In [82]:
#dropped columns that were not relevant to model
jokic_df.drop(columns= ["Rk","Tm","Date","Age","Opp"], axis=1, inplace = True)
jokic_df.head()
jokic_df.columns

Index(['G', 'Unnamed: 5', 'Unnamed: 7', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM'],
      dtype='object')

In [83]:
#added titles to columns without names
jokic_df.rename(columns={'Unnamed: 5':'Home_or_Away'},inplace= True)
jokic_df.rename(columns={'Unnamed: 7':'MOV'},inplace= True)

In [84]:
#changed Home or Away column so that if the game was at home it is a value of 0 and if away it is 1
jokic_df['Home_or_Away'] = jokic_df['Home_or_Away'].replace('@', 1)
jokic_df = jokic_df.fillna(0)
jokic_df

,G,Home_or_Away,MOV,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,0.333,2.0,2.0,1.000,2.0,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0,0.755,0.735,6.8,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
1,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,0.333,2.0,2.0,1.000,2.0,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0,0.488,0.500,19.1,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
2,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,0.000,1.0,3.0,0.333,5.0,9.0,14.0,6.0,1.0,0.0,1.0,6.0,13.0,14.2,15.0,0.755,0.735,6.8,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
3,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,0.000,1.0,3.0,0.333,5.0,9.0,14.0,6.0,1.0,0.0,1.0,6.0,13.0,14.2,15.0,0.488,0.500,19.1,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
4,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,0.500,11.0,11.0,1.000,3.0,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0,0.729,0.577,10.5,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
5,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,0.500,11.0,11.0,1.000,3.0,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0,0.588,0.553,11.1,21.5,16.3,35.0,0.0,0.0,11.6,31.6,126.0,125.0,6.0
6,2.0,0.0,W (+9),1,37:14,10.0,19.0,0.526,1.0,3.0,0.333,6.0,9.0,0.667,3.0,6.0,9.0,9.0,0.0,0.0,3.0,4.0,27.0,22.1,6.0,0.729,0.577,10.5,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
7,2.0,0.0,W (+9),1,37:14,10.0,19.0,0.526,1.0,3.0,0.333,6.0,9.0,0.667,3.0,6.0,9.0,9.0,0.0,0.0,3.0,4.0,27.0,22.1,6.0,0.588,0.553,11.1,21.5,16.3,35.0,0.0,0.0,11.6,31.6,126.0,125.0,6.0
8,3.0,0.0,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,1.000,6.0,9.0,0.667,1.0,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0,0.681,0.650,2.7,39.6,21.3,46.7,1.2,2.2,17.7,18.0,142.0,105.0,11.7
9,3.0,0.0,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,1.000,6.0,9.0,0.667,1.0,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0,0.769,0.769,8.1,27.7,19.3,41.3,1.3,0.0,23.5,20.7,137.0,113.0,10.2


In [85]:
#remove rows with games Jokic did not play
jokic_df = jokic_df[jokic_df['GS'] != "Inactive"]
jokic_df = jokic_df[jokic_df['GS'] != "Did Not Dress"]
jokic_df = jokic_df[jokic_df['GS'] != "Did Not Play"]
display(jokic_df)

,G,Home_or_Away,MOV,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,0.333,2.0,2.0,1.000,2.0,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0,0.755,0.735,6.8,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
1,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,0.333,2.0,2.0,1.000,2.0,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0,0.488,0.500,19.1,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
2,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,0.000,1.0,3.0,0.333,5.0,9.0,14.0,6.0,1.0,0.0,1.0,6.0,13.0,14.2,15.0,0.755,0.735,6.8,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
3,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,0.000,1.0,3.0,0.333,5.0,9.0,14.0,6.0,1.0,0.0,1.0,6.0,13.0,14.2,15.0,0.488,0.500,19.1,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
4,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,0.500,11.0,11.0,1.000,3.0,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0,0.729,0.577,10.5,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
5,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,0.500,11.0,11.0,1.000,3.0,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0,0.588,0.553,11.1,21.5,16.3,35.0,0.0,0.0,11.6,31.6,126.0,125.0,6.0
6,2.0,0.0,W (+9),1,37:14,10.0,19.0,0.526,1.0,3.0,0.333,6.0,9.0,0.667,3.0,6.0,9.0,9.0,0.0,0.0,3.0,4.0,27.0,22.1,6.0,0.729,0.577,10.5,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
7,2.0,0.0,W (+9),1,37:14,10.0,19.0,0.526,1.0,3.0,0.333,6.0,9.0,0.667,3.0,6.0,9.0,9.0,0.0,0.0,3.0,4.0,27.0,22.1,6.0,0.588,0.553,11.1,21.5,16.3,35.0,0.0,0.0,11.6,31.6,126.0,125.0,6.0
8,3.0,0.0,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,1.000,6.0,9.0,0.667,1.0,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0,0.681,0.650,2.7,39.6,21.3,46.7,1.2,2.2,17.7,18.0,142.0,105.0,11.7
9,3.0,0.0,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,1.000,6.0,9.0,0.667,1.0,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0,0.769,0.769,8.1,27.7,19.3,41.3,1.3,0.0,23.5,20.7,137.0,113.0,10.2


In [86]:
#convert minutes played column to seconds played 
def convert_to_numerical(time_obj):
    minutes, seconds = time_obj.split(':')
    total_seconds = int(minutes) * 60 + int(seconds)
    return total_seconds

jokic_df['MP'] = jokic_df['MP'].apply(convert_to_numerical)
jokic_df.rename(columns={'MP':'Seconds_played'},inplace =True)
jokic_df.dtypes

G                 float64
Home_or_Away      float64
MOV                object
GS                 object
Seconds_played      int64
FG                float64
FGA               float64
FG%               float64
3P                float64
3PA               float64
3P%               float64
FT                float64
FTA               float64
FT%               float64
ORB               float64
DRB               float64
TRB               float64
AST               float64
STL               float64
BLK               float64
TOV               float64
PF                float64
PTS               float64
GmSc              float64
+/-               float64
TS%               float64
eFG%              float64
ORB%              float64
DRB%              float64
TRB%              float64
AST%              float64
STL%              float64
BLK%              float64
TOV%              float64
USG%              float64
ORtg              float64
DRtg              float64
BPM               float64
dtype: objec

In [87]:
jokic_df.columns

Index(['G', 'Home_or_Away', 'MOV', 'GS', 'Seconds_played', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM'],
      dtype='object')

In [88]:
#converted columns with float64 types to int64 types
jokic_int_df = jokic_df[['G', 'Home_or_Away','FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM']].round().astype('int64')


In [89]:
#concat dataframes that contained all columns
jokic_df.drop(columns= ['G', 'Home_or_Away','FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM'], axis=1, inplace = True)
jokic_full_df = pd.concat([jokic_df, jokic_int_df], axis=1)
jokic_full_df.tail()

,MOV,GS,Seconds_played,G,Home_or_Away,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
577,L (-14),1,1662,76,1,4,10,0,0,3,0,2,3,1,2,3,5,6,4,0,6,1,10,8,-10,0,0,6,13,9,36,6,0,35,24,73,104,-2
578,W (+28),1,1720,77,0,9,10,1,0,1,0,2,3,1,0,11,11,9,0,0,5,3,20,20,18,1,1,0,37,22,47,0,0,31,26,125,87,12
579,W (+9),1,2205,78,0,9,15,1,1,4,0,3,4,1,3,10,13,9,2,1,2,3,22,26,6,1,1,10,34,22,38,3,2,11,23,139,111,13
581,L (-10),1,963,79,1,1,6,0,0,0,0,0,2,0,2,3,5,2,1,1,1,6,2,-1,-15,0,0,14,24,19,16,3,6,13,22,57,112,-15
582,W (+4),1,2195,80,0,13,26,0,1,7,0,2,2,1,3,11,14,2,3,0,4,4,29,20,16,1,1,9,30,20,12,4,0,13,38,100,93,4


In [90]:
#changed MOV to win or loss column and another with actual margin
jokic_full_df['WinOrLoss'] = jokic_full_df['MOV'].str[0].map({'L':0,'W':1})
jokic_full_df['Margin'] = jokic_full_df['MOV'].str[1:].str.extract("(\d+)").astype(int)
jokic_full_df.dtypes

MOV               object
GS                object
Seconds_played     int64
G                  int64
Home_or_Away       int64
FG                 int64
FGA                int64
FG%                int64
3P                 int64
3PA                int64
3P%                int64
FT                 int64
FTA                int64
FT%                int64
ORB                int64
DRB                int64
TRB                int64
AST                int64
STL                int64
BLK                int64
TOV                int64
PF                 int64
PTS                int64
GmSc               int64
+/-                int64
TS%                int64
eFG%               int64
ORB%               int64
DRB%               int64
TRB%               int64
AST%               int64
STL%               int64
BLK%               int64
TOV%               int64
USG%               int64
ORtg               int64
DRtg               int64
BPM                int64
WinOrLoss          int64
Margin             int64


In [91]:
#dropped MOV and GS
jokic_full_df.drop(columns= ["MOV","GS","G"], axis=1, inplace = True)

In [92]:
display(jokic_full_df)

,Seconds_played,Home_or_Away,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,WinOrLoss,Margin
0,2005,1,12,17,1,1,3,0,2,2,1,2,2,4,6,3,0,3,4,27,24,-5,1,1,7,8,7,38,4,0,14,27,136,119,14,0,21
1,2005,1,12,17,1,1,3,0,2,2,1,2,2,4,6,3,0,3,4,27,24,-5,0,0,19,32,26,33,2,0,7,22,124,83,6,0,21
2,1709,0,6,12,0,0,0,0,1,3,0,5,9,14,6,1,0,1,6,13,14,15,1,1,7,8,7,38,4,0,14,27,136,119,14,1,29
3,1709,0,6,12,0,0,0,0,1,3,0,5,9,14,6,1,0,1,6,13,14,15,0,0,19,32,26,33,2,0,7,22,124,83,6,1,29
4,2064,1,7,13,1,1,2,0,11,11,1,3,9,12,10,0,0,5,4,26,25,-3,1,1,10,35,22,38,0,0,22,27,138,119,7,1,5
5,2064,1,7,13,1,1,2,0,11,11,1,3,9,12,10,0,0,5,4,26,25,-3,1,1,11,22,16,35,0,0,12,32,126,125,6,1,5
6,2234,0,10,19,1,1,3,0,6,9,1,3,6,9,9,0,0,3,4,27,22,6,1,1,10,35,22,38,0,0,22,27,138,119,7,1,9
7,2234,0,10,19,1,1,3,0,6,9,1,3,6,9,9,0,0,3,4,27,22,6,1,1,11,22,16,35,0,0,12,32,126,125,6,1,9
8,2321,0,6,10,1,1,1,1,6,9,1,1,15,16,13,1,1,3,2,19,25,18,1,1,3,40,21,47,1,2,18,18,142,105,12,1,5
9,2321,0,6,10,1,1,1,1,6,9,1,1,15,16,13,1,1,3,2,19,25,18,1,1,8,28,19,41,1,0,24,21,137,113,10,1,5


In [48]:
y = jokic_full_df['PTS'].values
display(y)

array([27, 27, 13, 13, 26, 26, 27, 27, 19, 19, 20, 20,  9,  9, 43, 43, 31,
       31, 28, 28, 12, 12, 24, 24, 23, 23, 39, 39, 15, 15, 30, 30, 21, 21,
       53, 53, 26, 26, 29, 29, 24, 24, 32, 32, 29, 29, 34, 34,  8,  8, 23,
       23, 24, 24, 30, 30, 27, 27, 31, 39, 19, 32, 17, 24, 32, 19, 33, 31,
       43, 25, 40, 13, 29, 41, 20, 40, 19, 30, 24, 12, 28, 14, 21, 17, 36,
       31, 25, 24, 26, 22, 14, 20, 29, 30, 27, 14, 24, 15, 40, 14, 18, 17,
       18, 37, 35, 28, 30, 24, 22, 31, 31, 25, 14,  6, 27, 27, 25, 25, 32,
       32, 26, 26, 24, 24, 37, 37, 24, 24, 37, 37, 11, 11, 30, 30, 26, 23,
       34, 28, 25, 22, 28, 35, 30, 24, 18, 32, 17, 39, 22, 35, 28, 27, 20,
       13, 29, 26, 22, 24, 27, 26, 33, 22, 21, 20, 17, 25, 49, 26, 34, 28,
       26, 29, 18, 21, 25, 27, 21, 23, 28, 26, 35, 25, 18,  8, 22, 46, 32,
       38, 23, 26, 22, 29, 32, 23, 30, 28, 35, 26, 37, 38, 38, 41, 35, 29,
       29, 34, 34, 24, 24, 38, 38, 19, 19, 36, 36, 26, 26, 16, 16, 17, 17,
       38, 38, 19, 19, 36

In [49]:
X = jokic_full_df.drop(columns= 'PTS').values
display(X)

array([[2005,    1,    1, ...,   14,    0,   21],
       [2005,    1,    1, ...,    6,    0,   21],
       [1709,    1,    0, ...,   14,    1,   29],
       ...,
       [2205,   78,    0, ...,   13,    1,    9],
       [ 963,   79,    1, ...,  -15,    0,   10],
       [2195,   80,    0, ...,    4,    1,    4]])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [51]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [63]:
number_input_features = 37

In [64]:
number_output_neurons = 1

In [65]:
hidden_nodes_layer1 = 8

In [66]:
hidden_nodes_layer2 = 4

In [67]:
nn = Sequential()

In [68]:
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [69]:
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [70]:
nn.add(Dense(units=1, activation="linear"))

In [71]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 304       
                                                                 
 dense_4 (Dense)             (None, 4)                 36        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 345
Trainable params: 345
Non-trainable params: 0
_________________________________________________________________


In [72]:
nn.compile(loss= "mse", optimizer="adam", metrics=["mse"])

In [73]:
jokic_deep_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
14/14 [==============================] - 1s 2ms/step - loss: 682.5836 - mse: 682.5836
Epoch 2/50
14/14 [==============================] - 0s 2ms/step - loss: 675.2139 - mse: 675.2139
Epoch 3/50
14/14 [==============================] - 0s 4ms/step - loss: 669.4971 - mse: 669.4971
Epoch 4/50
14/14 [==============================] - 0s 3ms/step - loss: 665.4904 - mse: 665.4904
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 662.2156 - mse: 662.2156
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 659.6040 - mse: 659.6040
Epoch 7/50
14/14 [==============================] - 0s 2ms/step - loss: 657.4622 - mse: 657.4622
Epoch 8/50
14/14 [==============================] - 0s 2ms/step - loss: 655.5721 - mse: 655.5721
Epoch 9/50
14/14 [==============================] - 0s 5ms/step - loss: 653.9339 - mse: 653.9339
Epoch 10/50
14/14 [==============================] - 0s 3ms/step - loss: 652.4960 - mse: 652.4960
Epoch 11/50
14/14 [==========

In [74]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

5/5 - 0s - loss: 597.2605 - mse: 597.2605 - 156ms/epoch - 31ms/step


In [75]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 597.260498046875, Accuracy: 597.260498046875


In [78]:
predictions = (nn.predict(X_test_scaled) > 0.5).astype("int32")

5/5 [==============================] - 0s 4ms/step
